In [ ]:
import qutip as qt
import qiskit as qs
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

from quocslib.Optimizer import Optimizer
from quocslib.utils.AbstractFoM import AbstractFoM

In [ ]:
def treat_u_y(u_y, tlist):
    def check_len():
        if len(u_y) != len(tlist):
            raise ValueError("Mismatched pulse and time bins.")
    
    if isinstance(u_y, np.ndarray):
        check_len()
        return u_y
    elif isinstance(u_y, list):
        check_len()
        return np.array(u_y)
    elif callable(u_y):
        return u_y(tlist)
    else:
        return np.array([u_y]*len(tlist))
    

def get_fidelity(u_y,
                 N_sample = 100,
                 time_steps = 501,
                 pulse_duration = 1,
                 noise_strength = 20,
                 plot_bloch = False):

    rho_target = (qt.basis(2,0) + qt.basis(2,1)).unit()

    tlist = np.linspace(0, pulse_duration, time_steps)

    u_y = treat_u_y(u_y, tlist)

    rho_t = [0]*time_steps

    rng = np.random.default_rng(seed=12345)
    for i in range(N_sample):
        white_noise = rng.normal(loc=0, scale=noise_strength, size=time_steps)
        
        H = [[qt.sigmaz()/2, white_noise],
             [qt.sigmay()/2, u_y]]
        
        rho_t_per_sample = qt.mesolve(H = H,
                                      tlist = tlist,
                                      rho0 = qt.basis(2, 0)).states
        
        for j in range(time_steps):
            rho_t[j] += rho_t_per_sample[j] / N_sample
        
    if plot_bloch:
        b = qt.Bloch()
        b.add_states(rho_t[1:-1], kind = "point", colors = ["b"]*(time_steps-2), alpha = 0.1)
        b.add_states(rho_t[0])
        b.add_states(rho_t[-1])
        b.show()
        
    return qt.fidelity(rho_t[-1], rho_target)

In [ ]:
qr = qs.QuantumRegister(1, "q")
q, = qr

qc = qs.QuantumCircuit(qr)

qc.h(q)

qc.draw("mpl")

In [ ]:
rho0 = qt.basis(2, 0)

H = qt.sigmay() * np.pi/4
tlst = np.linspace(0, 1, 101)

rhot = qt.mesolve(H, rho0, tlst).states

b = qt.Bloch()
b.add_states(rhot[1:-1], kind = "point", colors = ["b"]*99, alpha = 0.1)
b.add_states(rhot[0])
b.add_states(rhot[-1])
b.show()

In [ ]:
rho0 = qt.basis(2, 0)

tlst = np.linspace(0, 1, 101)

rhots = []

rng = np.random.default_rng(seed=12345)

for i in range(100):
    H = [[qt.sigmay()/2, np.full((101,), np.pi/2)],
         [qt.sigmaz()/2, rng.normal(loc=0, scale=20, size=101)]]

    rhot = qt.mesolve(H, rho0, tlst).states
    
    if i == 0:
        rhots.extend(rhot)
    else:
        for j in range(101):
            rhots[j] += rhot[j]
    
for i in range(101):
    rhots[i] = rhots[i]/100

b = qt.Bloch()
b.add_states(rhots[1:-1], kind = "point", colors = ["b"]*99, alpha = 0.1)
b.add_states(rhots[0])
b.add_states(rhots[-1])
b.show()